In [42]:
import multiprocessing
import sys
import random
import os
import pickle
import operator
import collections
import numpy as np
import time


nchunk = 400
mode = 3#0: extract abs, 1: train_model, 2:generate sent
server = 'timan107'#'grenache''sherlock'
if server == 'timan107':
    pubmed_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/data/pubmed/'
    GO_data_dir = '/srv/local/work/swang141/Sheng_repo/data/Pubmed/'
    GO_term_file = '/srv/local/work/swang141/Sheng_repo/data/NLP_Dictionary/go_term.txt'
    repo_dir = '/srv/local/work/swang141/Sheng_repo/'
elif server == 'grenache':
    pubmed_dir = '/data/cellardata/users/majianzhu/wangsheng/Sheng_repo/pubmed/'
    GO_data_dir = '/cellar/users/majianzhu/Data/wangsheng/Sheng_repo/data/Pubmed/'
    GO_term_file = '/data/cellardata/users/majianzhu/wangsheng/Sheng_repo/Sheng_repo/data/NLP_Dictionary/go_term.txt'
    repo_dir = '/data/cellardata/users/majianzhu/wangsheng/Sheng_repo/Sheng_repo/'
else:
    pubmed_dir = '/oak/stanford/groups/rbaltman/swang91/Sheng_repo/data/Pubmed/pubmed/'
    GO_data_dir = '/oak/stanford/groups/rbaltman/swang91/Sheng_repo/data/Pubmed/'
    GO_term_file = '/oak/stanford/groups/rbaltman/swang91/Sheng_repo/data/NLP_Dictionary/go_term.txt'
    repo_dir = '/oak/stanford/groups/rbaltman/swang91/Sheng_repo/'



sys.path.append(repo_dir)
os.chdir(repo_dir)

from pikachu.datasets.WordNet import parse_sentence
from pikachu.models.text_classification.TextClassify import TextClassify
from sklearn.linear_model import LogisticRegression


GO_text_dir = GO_data_dir+'GO_abstracts_test/'
GO_pred_dir = GO_data_dir + 'GO_models_L2/'
GO_sent_dir = GO_data_dir + 'GO_sentences_L2/'
GO_sent_merge_dir = GO_data_dir + 'GO_sentences_L2_all/'

if not os.path.exists(GO_sent_merge_dir):
    os.makedirs(GO_sent_merge_dir)
if not os.path.exists(GO_sent_dir):
    os.makedirs(GO_sent_dir)
if not os.path.exists(GO_pred_dir):
    os.makedirs(GO_pred_dir)
if not os.path.exists(GO_text_dir):
    os.makedirs(GO_text_dir)


sent_GO_term = set()
fin = open(GO_term_file)
ct = 0
GO2auc = {}
GO2key = {}
GO2
auc_l = []
for line in fin:
    GO = line.lower().strip()
    log_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.log'
    if not os.path.isfile(log_file):
        continue
    GO2key[GO] = set()
    flog = open(log_file)
    for line in flog:
        w = line.strip().split('\t')
        auc = float(w[0])
        
        if not np.isnan(auc) and w[1].isdigit():
            auc_l.append(auc)
            GO2auc[GO] = auc
        if not w[1].isdigit():
            GO2key[GO].add(w[1])
    flog.close()
    #break
fin.close()




In [35]:
auc_l = np.array(auc_l)
for thres in [0.9,0.95,0.99]:
    ngo = len(np.where(auc_l > thres)[0])
    print thres, ngo *1.0 / len(auc_l)

0.9 0.599942062572
0.95 0.46567207416
0.99 0.141077636153


In [25]:
auc_sort = sorted(GO2auc.items(), lambda x, y: cmp(x[1], y[1]), reverse=True)


In [44]:
for i in auc_sort:
    print i,GO2key[i[0]]

('response to acrylamide', 1.0) set(['1.0', 'over', 'vitro', 'filaments', 'loeds', 'data', 'intracellular', 'acrylamide', 'gfap', 'cytostatic', 'levels', 'oxidative', 'keratin', 'u373-mg', 'exposure', 'stress', 'glutathione', 'phosphorylation', 'acrylamide.', 'were', 'gsh', 'toxicity'])
('response to furfural', 1.0) set(['1.0', 'dynamic', 'furfural', 'as', 'saccharomyces', 'acid', 'nadph-dependent', 'same', 'inhibitors', 'nadph', 'production', 'aldehyde', 'films', 'reductase', 'yeast', 'genes', 'ethanol', 'cerevisiae', 'stress', 'lignocellulosic', 'ygl157w', 'flux', 'model', 'gene', 'fermentation'])
('dehydroascorbic acid transport', 1.0) set(['1.0', 'dehydroascorbic', 'radical', 'recycling', 'acid', 'transport', 'cll', 'intracellular', 'dha', 'reduction', 'was', 'across', 'glucose', 'blood-retinal', 'normal', '-brain', 'retina', 'vitamin', 'free', 'ascorbic', 'transporter', 'ascorbate', 'glutathione', 'uptake'])
('ferrous iron transport', 1.0) set(['copper', 'domain', '1.0', 'sirt2', 

('n-glycan processing', 0.9926397205588822) set(['glycoprotein', '1.0', 'glycans', 'glycoproteins', 'inhibitors', 'n-glycans', 'alpha-glucosidase', 'alpha-mannosidase', 'n-glycosylation', 'golgi', 'enzyme', 'er', 'n-glycan', 'processing', 'e-cadherin', 'biosynthesis', 'glycan', 'n-linked', 'recombinant', 'glycosylation', 'enzymes', 'mannose'])
('heart valve development', 0.992636815920398) set(['heart', '1.0', 'valvular', 'design', 'matrix', 'valves', 'valve', 'csa', 'role', 'hsct', 'zebrafish', 'dogs', 'cathepsin', 'mesenchyme', 'signaling', 'cushion', 'cardiac', 'these', 'endocardial', 'emt', 'vegf', 'notch', 'expression', 'replacement'])
('seed development', 0.992620154296801) set(['embryo', 'endosperm', 'mutant', 'characterization', 'seed', 'soybean', 'acid', 'maize', 'abscisic', 'storage', 'arabidopsis', 'barley', 'rice', 'development', 'plant', 'aba', 'genes', 'biosynthesis', 'during', 'plants', 'stages', 'developing', 'proteins', '0.9999855361379594', 'yield', 'seeds', 'gene', '

('euchromatin', 0.9826556885380413) set(['transcriptional', 'drosophila', 'cytoplasm', 'heterochromatic', '0.9999929312482329', 'chromosome', 'nucleus.', 'dna', 'nuclear', 'localization', 'regions', 'genome', 'membrane', 'ultrastructure', 'chromatin', 'epigenetic', 'chromosomes', 'histone', 'heterochromatin', 'nucleus', 'euchromatic', 'y', 'nuclei', 'heterochromatin.'])
('gamma-tubulin ring complex', 0.9826039684696928) set(['gamma-tubulin', 'mutations', 'gamma-turc', 'mitosis', 'at', 'array', 'spindle', 'gcp6', 'microtubule', 'nucleation', 'complex', 'centrosomes', 'gammaturc', 'microtubules', 'fam190a', 'nedd1', 'centrosome', 'centrosomal', 'tubulin', '0.9999450035747677', 'tacc3', 'mt', 'mts', 'organization'])
('thylakoid', 0.9825962800875273) set(['synechocystis', 'chloroplast', 'chlorophyll', 'membranes', 'membranes.', 'cyanobacteria', '0.9999849397590361', 'chloroplasts', 'plastid', 'arabidopsis', 'lumen', 'membrane', 'plant', 'plastids', 'genes', 'proton', 'membrane.', 'plants',

('beta-galactosidase activity', 0.9664320345616815) set(['lysosomal', 'beta-galactosidase', 'senescence', 'fusions', 'of.', 'binding', 'growth', 'coli', 'lacz', '0.9999998583734703', 'yeast', 'hybrid', 'beta-galactosidase.', 'induction', 'assay', 'reporter', 'fusion', 'growth.', 'escherichia', 'beta-glucosidase', 'lactose', 'senescence-associated', 'gm1'])
('vasomotion', 0.9664187604690118) set(['arterial', 'diameter', 'endothelial', 'flow', 'microcirculatory', 'frequency', 'skin', 'oscillations', 'cardiovascular', 'arteries', 'perfusion', '0.9964464544852558', 'physiological', 'vessels', 'coronary', 'microcirculation', 'vascular', 'doppler', 'endothelium', 'pressure', 'blood', 'artery', 'capillary', 'arteriolar', 'microvascular', 'arterioles', 'smooth'])
('response to simvastatin', 0.9664026402640264) set(['endothelial', '1.0', 'p85', 'eif2alpha', 'treatment.', 'alpha-subunit', 'statin', 'cardiovascular', 'treatment', 'allele', 'simvastatin', 'plasma', 'ldl-c', 'apoptosis', 'after', '

('chlamydospore formation', 0.9340463458110517) set(['mutant', '1.0', 'albicans', 'wall', 'macroconidia', 'fungal', 'f.', 'fusarium', 'germ', 'phosphatase', 'mutants', 'albicans.', 'medium', 'c.', 'yeast', 'alkaline', 'hog1', 'candida', 'chlamydospores', 'on', 'lucidum', 'liquid', 'oxysporum', 'strains'])
('d4 dopamine receptor binding', 0.9339999999999999) set(['1.0', 'defined', 'binding', 'brain', '3hnemonapride..', 'receptor', 'measure', '3hnemonapride', 'affinities', 'cerebellum.', 'd4', 'as', 'antipsychotic', 'had', 'by', 'only', 'sulpiride.', 'drugs', 'frontal', 'component', 'cloned', 'subtraction', '3hraclopride', 'ligands', 'striatum', 'cow', 'atypical', 'rat', 'cortex', 'typical', 'sigma', 'minus', 'labelled'])
('ego complex', 0.9339999999999999) set(['rag', '1.0', 'ego1', 'ego3', 'proliferation', 'nitrogen', 'amino', 'ego-tc', 'vacuolar', 'recovery', 'tor', 'acts', 'complex', 'egoc', 'rapamycin', 'yeast', 'residual', 'torc1', 'signaling', 'gln3', 'structural', 'gtr2', 'gtr1',

('respiratory burst at fertilization', 0.874) set(['domain', 'organification', '1.0', 'phagocyte', 'duox1', 'duox2', 'studies;', 'surface', 'duox', 'dual', 'defect"', 'hydrogen', 'urchin', 'thyroid', 'based', 'fertilization', 'udx1', 'hormonogenesis', 'peroxidase.', '"total', 'synthesis', 'generating', 'nadph-oxidase', 'evolves', 'homologous', 'gp91', 'oxidase', 'both', 'phox', 'peroxide', 'duoxes'])
('er membrane protein complex', 0.874) set(['multi-pass', 'erad', '1.0', 'rh1', 'is', 'mutants', 'mitochondrial', 'peptide', 'sec72p', 'interacts', 'subject', 'sec72', 'sr7575', 'slc25a46', 'nuclear', 'transmembrane', 'complex', 'autophagosome', 'membrane', 'kar2p', 'er', 'translocation', 'rab5a', 'that', 'reticulum', 'fusion', 'e3', 'dpobemc3', 'for', 'emc', 'endoplasmic', 'required', 'karyogamy'])
('embryonic eye morphogenesis', 0.8739205526770294) set(['development.', '1.0', 'klf5cn', 'number', 'surface', 'as', 'chf1hey2', 'serum', 'layers', 'eye', 'defective', 'factors', 'provide', 'so

('histone h4-k16 acetylation', 0.6573498964803313) set(['1.0', 'into', 'functional', 'h4', 'inhibits', 'directly', 'transcription', 'negative', 'mark', 'h4-k16ac', 'role', 'sir2delta', 'modification', 'chromatin', 'ability', 'histone', 'sir2', 'mechanistic', 'structure', 'both', 'introduced', 'euchromatic', 'acetylation', 'this', 'plasmid', 'domains', 'promotion', 'gene'])
('regulation of torc1 signaling', 0.6570796460176992) set(['mutant', '1.0', 'gy', 'kinase', 'ypk3', 'radiation', 's6', 'selumetinib', 'rapamycin-sensitive', 'xenografts', 'xrt', 'context', 'yeast', 'effect', 'mek', 'torc1', 'inhibition', 'signaling', 'bt-40', 'suppressed', 'braf-mutant', 'phosphorylation', 'anaplastic', 'rps6', 'hypersensitivity', 'braf', 'fancd2', 'childhood'])
('growth hormone activity', 0.6570048309178744) set(['1.0', 'ga3', 'ghmpk11', 'growth', 'human', 'igf-i', 'gh', 'homocysteine', 'group', 'factors', 'of.', 'serum', 'hgh', 'mab-ebl', 'monoclonal', 'hormone', 'assay', 'igf-1', 'some', 'and.', '

('indolepyruvate decarboxylase activity', 0.0) set(['indole-3-pyruvic', 'gene', 'harboring', 'strains', 'aminotransferase', 'decarboxylase', 'enterobacter', 'biosynthesis.', 'indole-3-acetaldehyde', 'same', 'produced', 'amounts', 'increases', 'experiments', 'indolepyruvate', 'l-tryptophan', 'pseudomonas', 'indole-3-acetic', 'acid', 'those'])
('non-membrane-bounded organelle', 0.0) set(['dried', 'functions', 'proteins.', 'binding', 'mainly', 'ribosome', 'genes', 'proteins', 'identified', 'analysis', 'dragon', 'enriched', 'pathways', 'sea', 'were', 'using', 'protein', 'gene', 'related', 'vte.'])
('striated muscle tissue development', 0.0) set(['sheep', 'development', 'different', 'number', 'putative', 'study', 'non-coding', 'tissues', 'rnas', 'length', 'long', 'prediction', 'lncrnas', 'important', 'eight', 'tissues-specific', 'lincrnas', 'using', 'intergenic', 'genome'])
('phytochelatin transport', 0.0) set(['plants', 'abc', 'mediates', 'abc2', 'organisms', 'ers', 'quintuple', 'into', 's

In [43]:
GO2keyMicroneme

{'triplex dna binding': {'1.0',
  'assembly',
  'at',
  'based',
  'binders..',
  'change',
  'color',
  'colorimetrically',
  'dna',
  'helixes',
  'melting',
  'molecules',
  'nanoparticle',
  'nanoparticles.',
  'only',
  'presence',
  'process',
  'red-to-blue',
  'screening',
  'significantly',
  'structure',
  'triple',
  'triplex'},
 'thymidine kinase activity': {'1.0',
  'acyclovir',
  'antiviral',
  'assay',
  'azt',
  'cell',
  'cells',
  'cellular',
  'enzyme',
  'hepatectomy',
  'hepatic',
  'herpes',
  'hsv-1',
  'incorporation',
  'kinase',
  'mutants',
  'prognostic',
  'rats',
  'regeneration',
  'resistant',
  'simplex',
  'thymidine',
  'thymidylate',
  'tk',
  'tk1',
  'zidovudine'},
 'transcription coactivator activity': {'1.0',
  '24-week-old',
  '6-week-old',
  'activity',
  'analysis',
  'cells',
  'complex',
  'deficits',
  'drosophila',
  'dysfunction',
  'expression',
  'first',
  'identified',
  'imd',
  'immune',
  'improved',
  'marrow',
  'mice',
  'pathwa